In [10]:
using Globtim
using CSV
using DataFrames
include("../src/lib_func.jl")

# Load the dataframe from the CSV file
df_2d = CSV.read("../data/camel_d6.csv", DataFrame)

# Constants and Parameters
d = 3      # Degree 
const n, a, b = 4, 5, 1 
const scale_factor = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .1 , 2 / 10  # Sampling parameters
const tol_l2 = 1e-1               # Define the tolerance for the L2-norm
# const tol_l2 = 1e-0
# The objective function
f = camel_4d 
# f = camel_3_by_3 # Objective function

camel_4d (generic function with 1 method)

In [11]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, n, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        global d += 1
    end
end

current L2-norm: 23.606406291129833
Number of samples: 7
current L2-norm: 2.4807257531880382
Number of samples: 9
current L2-norm: 1.326509954507463
Number of samples: 12
attained the desired L2-norm: 6.834956284109819e-14


In [12]:
# Assuming x is already defined as @polyvar x[1:2]
loc = "inputs.ms"
# File path of the output file
file_path_output = "outputs.ms";

In [13]:
using DynamicPolynomials, DataFrames
ap = main_nd(n, d, poly_approx.coeffs)
@polyvar(x[1:n]) # Define polynomial ring 
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x
names = [x[i].name for i in 1:length(x)]
open(loc, "w") do file
    println(file, join(names, ", "))
    println(file, 0)
end
# Define the polynomial approximant 
PolynomialApproximant = sum(ap .* MonomialVector(x, 0:d))
for i in 1:n
    partial = differentiate(PolynomialApproximant, x[i])
    partial_str = replace(string(partial), "//" => "/")
    open(loc, "a") do file
        if i < n
            println(file, string(partial_str, ","))
        else
            println(file, partial_str)
        end
    end
end

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [14]:
run(`msolve -v 1 -f inputs.ms -o outputs.ms`)


--------------- INPUT DATA ---------------
#variables                       4
#equations                       4
#invalid equations               0
field characteristic             0
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                         1
info level                       1
generate pbm files               0
------------------------------------------
Dimension of quotient: 625
[625, 85], Non trivial / Trivial = 13.60%
Density of non-trivial part 91.70%
Time spent to generate sequence (elapsed): 0.04 sec (1.80 Gops/sec)
Time spent to compute eliminating polynomial (elapsed): 0.00 sec
Elimination polynomial has degree 625.

Starts multi-modular computations
{2}{4}{8}{16}{32}{64}{128}{256}{512}
<Step:2/0.02/0.08>{1024}
<Step:4/


---------------- TIMINGS ---------------
overall(elapsed)        0.05 sec
overall(cpu)            0.05 sec
select                  0.00 sec   8.6%
symbolic prep.          0.02 sec  32.4%
update                  0.00 sec   2.9%
convert                 0.00 sec   6.9%
linear algebra          0.02 sec  46.9%
reduce gb               0.00 sec   0.0%
-----------------------------------------

---------- COMPUTATIONAL DATA -----------
size of basis                   108
#terms in basis               55621
#pairs reduced                  297
#GM criterion                  5808
#redundant elements               3
#rows reduced                   702
#zero reductions                190
max. matrix data                390 x 990 (39.294%)
max. symbolic hash table size  2^11
max. basis hash table size     2^16
-----------------------------------------



Elapsed time (real root extraction) = 186.34
------------------------------------------------------------------------------------
msolve overall time         859.68 sec (elapsed) / 859.29 sec (cpu)
------------------------------------------------------------------------------------


Process(`msolve -v 1 -f inputs.ms -o outputs.ms`, ProcessExited(0))

Construct a Dataframe with only the critical points of the approximant $w_d$ which fall into the $[-1, 1]^4$ domain and rescale them to the original domain. 

In [15]:
function average(X::Vector{Int})::Float64
    return sum(X) / length(X)
end

# Process the file and get the points
evaled = process_output_file(file_path_output)

# Parse the points into correct format
real_pts = []
for pts in evaled
    if typeof(pts) == Vector{Vector{Vector{BigInt}}}
        X = parse_point(pts)
    else
        X = average.(pts)
    end
    push!(real_pts, Float64.(X))
end

condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1 && -1 < point[3] < 1 && -1 < point[4] < 1 
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x1 = scale_factor * Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_x2 = scale_factor * Float64[point[2] for point in filtered_points] 
h_x3 = scale_factor * Float64[point[3] for point in filtered_points] 
h_x4 = scale_factor * Float64[point[4] for point in filtered_points] 

height = map(p -> f(p), zip(h_x1, h_x2, h_x3, h_x4)) # Compute the height of the critical points
df = DataFrame(x1 = h_x1, x2 = h_x2, x3 = h_x3, x4 = h_x4, height = height); # Create a DataFrame

Now we want to compute the distance between the critical points of the 2d function.

The true "exact" critical points of the 2d `Camel` (6 humps) function are stored in `df_2d`. 

We compute which of those are the closest to the `[1:2]` and `[3:4]` coordinates of the critical points of our 4d approximant.  

The true critical points are stored in `df_2d`. Need to split to distance to minima first coordinate and second set of coordinates ten combine them.


In [16]:
using LinearAlgebra

total_distance = Float64[]
for i in 1:nrow(df)
    distances_1 = [norm([df.x1[i], df.x2[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    distances_2 = [norm([df.x3[i], df.x4[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    # Compute distances to all points in df_2d
    min_value_1, min_index_1 = findmin(distances_1)
    min_value_2, min_index_2 = findmin(distances_2) 

    # Extract the corresponding points
    point_1 = [df.x1[i], df.x2[i], df.x3[i], df.x4[i]]
    point_2 = [df_2d.x[min_index_1], df_2d.y[min_index_1], df_2d.x[min_index_2], df_2d.y[min_index_2]]

    # Compute total distance
    tot_dist = norm(point_1 - point_2)
    push!(total_distance, tot_dist)

end
df.total_distance = total_distance
sorted_df = sort(df, :total_distance)


Row,x1,x2,x3,x4,height,total_distance
,Float64,Float64,Float64,Float64,Float64,Float64
1,1.63807,0.228674,1.70361,-0.796084,2.01389,2.6282e-13
2,1.23023,0.162335,1.70361,-0.796084,2.28083,2.6736e-13
3,-1.70361,0.796084,1.70361,-0.796084,-0.430928,2.70774e-13
4,-1.10921,0.768268,1.70361,-0.796084,0.328255,2.70776e-13
5,-0.089842,0.712656,1.70361,-0.796084,-1.24709,2.70814e-13
6,1.29607,0.605084,1.70361,-0.796084,2.01401,2.75723e-13
7,1.6071,0.568651,1.70361,-0.796084,1.88879,2.81017e-13
8,-2.27304e-14,1.04609e-13,1.70361,-0.796084,-0.215464,2.82317e-13
9,-0.089842,0.712656,1.10921,-0.768268,-0.48791,2.8313e-13


In [17]:
using Optim
df.steps = zeros(nrow(df))
df.converged = zeros(nrow(df))

for i in 1:nrow(df)
    # println("Optimizing for point $i")
    x0 = [df.x1[i], df.x2[i], df.x3[i], df.x4[i]]
    #  + 0.2 * randn(Float64, 4)
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=false))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)
    distance = norm(x0 - minimizer)
    # df.dist_to_loc_min[i] = distance
    df.steps[i] = steps
    df.converged[i] = converged
    # summary(res)
end
df

Row,x1,x2,x3,x4,height,total_distance,steps,converged
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.6071,-0.568651,1.70361,-0.796084,1.88879,4.05968e-13,0.0,1.0
2,-1.63807,-0.228674,1.70361,-0.796084,2.01389,3.76508e-13,0.0,1.0
3,1.70361,-0.796084,1.70361,-0.796084,-0.430928,2.84764e-13,0.0,1.0
4,-1.70361,0.796084,1.70361,-0.796084,-0.430928,2.70774e-13,0.0,1.0
5,1.63807,0.228674,1.70361,-0.796084,2.01389,2.6282e-13,0.0,1.0
6,-1.29607,-0.605084,1.70361,-0.796084,2.01401,3.57372e-13,0.0,1.0
7,1.6071,0.568651,1.70361,-0.796084,1.88879,2.81017e-13,0.0,1.0
8,-1.23023,-0.162335,1.70361,-0.796084,2.28083,3.2652e-13,0.0,1.0
9,1.10921,-0.768268,1.70361,-0.796084,0.328255,2.88605e-13,0.0,1.0
